In [ ]:
import numpy as np
import pandas as pd
import novosparc
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
nn = 5
alpha_linear = 0.8  
cell_nb = 3000
figsize = 50
alpha = 0.5 # plt opacity
hvg_path = 'raw/high_variable_genes_10day.txt'

### load & process dataset

In [ ]:
# data_path = 'raw/expr_matrix_10day_log_normalized.txt'   # 11566 cells and 2000 genes
# data_path = 'SCTransformed_10day_3000var.genes_pesudocell.txt'
data_path = 'raw/SCTransform_10day_pesudocell_matrix.txt'


# data_path = 'raw/zero_or_one.txt'  # DGE in the form of 0 or 1

output_folder = 'out'

In [ ]:
# load dataset:  expression matrix   col-gene,row-cell
dataset = novosparc.io.load_data(data_path)
dataset.shape

In [ ]:
# just to view gene names and the matrix
gene_names = np.asarray(dataset.var.index.tolist())

In [ ]:
# save the set to dataset.raw
dataset.raw = dataset
# select only genes in hvg_list
dataset, hvg = novosparc.pp.subset_to_hvg(dataset, hvg_file = hvg_path) # dataset[:,dataset.var.index.isin(gene_list)], gene_list
dataset.shape

### load fake target space (location_3d)

In [ ]:
location_3d = np.load('raw/fake_3d_tissue.npy')

In [ ]:
cell_nb = cell_nb
seed = np.random.choice(location_3d.shape[0], cell_nb,replace=False)
location_3d = location_3d[seed, :]

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = Axes3D(fig)
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])
x = location_3d[:,0]
y = location_3d[:,1]
z = location_3d[:,2]
ax.scatter(x,y,z,s=1)
plt.show()

### load epi (border_3d)

In [ ]:
atlas_epi = np.load('raw/epi_33119_atlas.npy')
atlas_ukn = np.load('raw/ukn_04921_atlas.npy')

In [ ]:
atlas_matrix = np.concatenate((atlas_epi, atlas_ukn), axis=1)
print(len(atlas_matrix[:,0]),len(atlas_matrix[:,1]))
print(sum(atlas_matrix[:,0]),sum(atlas_matrix[:,1]))

In [ ]:
atlas_matrix = atlas_matrix[seed]
print(sum(atlas_matrix[:,0]),sum(atlas_matrix[:,1]))

In [ ]:
### plot ukn
inner_loc = np.where(atlas_matrix[:,1].squeeze()!=0)
plt.figure(figsize=(3,1))
plt.scatter(location_3d[inner_loc, 1], location_3d[inner_loc, 0],s=1)
plt.title('inner_SMED30004921',fontsize = 'medium')
plt.xlim([-2,60])
plt.ylim([-2,32])
plt.show()

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = Axes3D(fig)
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])
x = location_3d[inner_loc,0]
y = location_3d[inner_loc,1]
z = location_3d[inner_loc,2]
ax.scatter(x,y,z,s=1,alpha=alpha)
plt.title('inner_SMED30004921',fontsize = 'xx-large')
plt.show()
print(np.max(x),np.min(x),np.max(y),np.min(y))

In [ ]:
### plot ukn
inner_loc = np.where(atlas_matrix[:,0].squeeze()!=0)
plt.figure(figsize=(3,1))
plt.scatter(location_3d[inner_loc, 1], location_3d[inner_loc, 0],s=1)
plt.title('epi_SMED30033119',fontsize = 'medium')
plt.xlim([-2,60])
plt.ylim([-2,32])
plt.show()

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = Axes3D(fig)
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])
x = location_3d[inner_loc,0]
y = location_3d[inner_loc,1]
z = location_3d[inner_loc,2]
ax.scatter(x,y,z,s=1,alpha=alpha)
plt.title('epi_SMED30033119',fontsize = 'xx-large')
plt.show()
print(np.max(x),np.min(x),np.max(y),np.min(y))

### Reconstruction
https://github.com/rajewsky-lab/novosparc/blob/master/reconstruct_drosophila_embryo_tutorial.ipynb

In [ ]:
#Reconstruct the tissue de novo/with marker
tissue = novosparc.cm.Tissue(dataset=dataset, locations=location_3d, output_folder=output_folder)
insitu_genes = ['SMED30033119','SMED30004921']

In [ ]:
gene_names = tissue.gene_names
markers_in_sc = [np.where(gene_names == item)[0][0] for item in insitu_genes]
markers_in_sc

In [ ]:
tissue.setup_reconstruction(markers_to_use=markers_in_sc, insitu_matrix=atlas_matrix, num_neighbors_s=nn, num_neighbors_t=nn)

In [ ]:
tissue.reconstruct(alpha_linear=alpha_linear, epsilon=5e-3)

In [ ]:
import time
t1 = time.time()
tissue.calculate_spatially_informative_genes()
t2 = time.time()
print(t2-t1)

### plot result

In [ ]:
sdge_hvg = tissue.sdge
sdge = tissue.calculate_sdge_for_all_genes()
sdge.shape

In [ ]:
gene_dict = {
    'SMED30011970':'eye and head',
    'SMED30030642':'pharynx',
    'SMED30001882':'brain and phx',
    'SMED30005457':'big cells-gut(++)',
    'SMED30000013':'gut',
    'SMED30010123':'protonephridia',
    'SMED30016244':'secretory cells',
    'SMED30011490':'epithelium'
}

In [ ]:
# plot some genes
gene_list_to_plot = [
    'SMED30011970', #eye and head, dd_4427
    'SMED30030642', #pharynx
    'SMED30001882',#brain and phx
    'SMED30005457', #super strong; big cells around the gut
    'SMED30000013', #gut
    'SMED30010123', #protonephridia
    'SMED30016244', #secretory cells?
    'SMED30011490', #epithelium     
]
list_marker = [g in tissue.gene_names for g in gene_list_to_plot]
list_index = np.nonzero(list_marker)[0]
gene_list_to_plot = [gene_list_to_plot[i] for i in list_index]

In [ ]:
mylen=len(gene_list_to_plot)
plt.figure(figsize=(2*mylen,6),facecolor='grey')

for i in range(len(gene_list_to_plot)):
        plt.subplot(1,mylen,i+1)        
        expr_val = sdge[np.argwhere(gene_names == gene_list_to_plot[i])[0], :].flatten()        
        plt.scatter(location_3d[:, 0], location_3d[:, 1], c=expr_val, s=3)
        plt.title(gene_dict[gene_list_to_plot[i]])
        plt.axis('off')
plt.show()

### show input markers's perdiction

In [ ]:
ig_marker = [ig in gene_names for ig in insitu_genes]
ig_index = np.nonzero(ig_marker)[0]
ig_list_to_plot = [insitu_genes[i] for i in ig_index]

In [ ]:
mylen=len(ig_list_to_plot) +1
plt.figure(figsize=(2*mylen,6),facecolor='grey')
for i in range(mylen -1):
        plt.subplot(1,mylen,i+1)        
        expr_val = sdge[np.argwhere(gene_names == ig_list_to_plot[i])[0], :].flatten()        
        plt.scatter(location_3d[:, 0], location_3d[:, 1], c=expr_val, s=3)
        plt.title(ig_list_to_plot[i])
        plt.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig,facecolor='k')
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])
expr_val = sdge[np.argwhere(gene_names == ig_list_to_plot[0])[0], :].flatten()
x = location_3d[:,0]
y = location_3d[:,1]
z = location_3d[:,2]
ax.scatter(x,y,z,c=expr_val,s=1)#
plt.show()
print(ig_list_to_plot[0])

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig,facecolor='k')
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])
expr_val = sdge[np.argwhere(gene_names == ig_list_to_plot[1])[0], :].flatten()
x = location_3d[:,0]
y = location_3d[:,1]
z = location_3d[:,2]
ax.scatter(x,y,z,c=expr_val,s=1)#
plt.show()
print(ig_list_to_plot[1])

### double check raw inputs

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig,facecolor='k')
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])

expr_val = atlas_matrix[:,0] #
x = location_3d[:,0]
y = location_3d[:,1]
z = location_3d[:,2]
ax.scatter(x,y,z,c=expr_val,s=1)#
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig,facecolor='k')
ax.set_xlim([0,figsize])
ax.set_ylim([0,figsize])
ax.set_zlim([0,figsize])

expr_val = atlas_matrix[:,1] #
x = location_3d[:,0]
y = location_3d[:,1]
z = location_3d[:,2]
ax.scatter(x,y,z,c=expr_val,s=1)#
plt.show()

In [ ]:
# to show status of the input atlas
mylen=len(ig_list_to_plot) +1
plt.figure(figsize=(2*mylen,6),facecolor='grey')
for i in range(mylen-1):
        plt.subplot(1,mylen,i+1)        
        expr_val = atlas_matrix[:,i]       
        plt.scatter(location_3d[:, 0], location_3d[:, 1], c=expr_val, s=3)
        plt.title(ig_list_to_plot[i])
        plt.axis('off')
plt.show()